In [1]:
import numpy as np
import pandas as pd

In [3]:
adult = pd.read_csv('adult.csv')
adult.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


response

In [9]:
adult = adult.assign(income = (adult['income'] == '>50k').astype(int))
adult.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,0
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,0
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,0
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,0
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,0


missing

In [11]:
adult = adult.replace('?', np.nan)
missing = [col for col in adult.columns if adult[col].isnull().any()]
missing

['workclass', 'occupation', 'native.country']

In [13]:
cat_cols = [col for col in adult.columns if adult[col].nunique()<10 and adult[col].dtype == 'object']
cat_cols

['workclass', 'marital.status', 'relationship', 'race', 'sex']

In [18]:
adult_red = adult.drop(missing, axis = 1)
adult_red.head()

,age,fnlwgt,education,education.num,marital.status,relationship,race,sex,capital.gain,capital.loss,hours.per.week,income
0,90,77053,HS-grad,9,Widowed,Not-in-family,White,Female,0,4356,40,0
1,82,132870,HS-grad,9,Widowed,Not-in-family,White,Female,0,4356,18,0
2,66,186061,Some-college,10,Widowed,Unmarried,Black,Female,0,4356,40,0
3,54,140359,7th-8th,4,Divorced,Unmarried,White,Female,0,3900,40,0
4,41,264663,Some-college,10,Separated,Own-child,White,Female,0,3900,40,0


handeling categoricals

In [24]:
cat_cols2 = [col for col in adult_red.columns if adult_red[col].nunique()<10 and adult_red[col].dtype == 'object']
cat_cols2

['marital.status', 'relationship', 'race', 'sex']

In [25]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoded = adult_red[cat_cols2].apply(encoder.fit_transform)

In [27]:
num_cols = [col for col in adult_red.columns if adult_red[col].dtype in ['int64','float64']]
num_cols

['age',
 'fnlwgt',
 'education.num',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'income']

In [29]:
adult_prim = adult_red[num_cols].join(encoded)
adult_prim.head()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,income,marital.status,relationship,race,sex
0,90,77053,9,0,4356,40,0,6,1,4,0
1,82,132870,9,0,4356,18,0,6,1,4,0
2,66,186061,10,0,4356,40,0,6,4,2,0
3,54,140359,4,0,3900,40,0,0,4,4,0
4,41,264663,10,0,3900,40,0,5,3,4,0


split

In [31]:
valid_fraction = 0.1
valid_size = int(len(adult_prim) * valid_fraction)
train = adult_prim[:-valid_size * 2]
valid = adult_prim[-valid_size * 2:-valid_size]
test = adult_prim[-valid_size:]

In [34]:
import lightgbm as lgb
from sklearn import metrics

In [37]:
feature_cols = train.columns.drop('income')

dtrain = lgb.Dataset(train[feature_cols], label=train['income'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['income'])

param = {'num_leaves': 64, 'objective': 'binary', 'metric': 'auc', 'seed': 7}
print("Training model!")
bst = lgb.train(param, dtrain, num_boost_round=1000, valid_sets=[dvalid], 
                early_stopping_rounds=10, verbose_eval=False)

valid_pred = bst.predict(valid[feature_cols])
valid_score = metrics.mean_absolute_error(valid['income'], valid_pred)
print(f"Validation AUC score: {valid_score:.4f}")

Training model!
Validation AUC score: 0.0000
